In [2]:
import pandas as pd
import numpy as np 

df=pd.read_csv("Shopify_data_challenge.csv",parse_dates=['created_at'])
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 04:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 04:35:11


In [3]:
df[['order_amount','total_items']].describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


#### We can see that there are large amounts of skewness detected by looking at the summary statistics of two columns in particular from our dataset, order_amount and total_items

In [14]:
## Let's look into average order totals greater than the 90th percentile across each shop which is $531
test_df=df.groupby('shop_id').agg({
    'user_id':'nunique','total_items':'count','order_amount':'mean'
})

## Renaming columns for convienience 
test_df.rename(columns = {'user_id':'Total Number of Unique Shoppers', 'total_items':'Total number of Items Sold',
                              'order_amount':'Avg. Total Order Amount'}, inplace = True)

test_df[test_df['Avg. Total Order Amount']>np.percentile(df['order_amount'],90)]

,Total Number of Unique Shoppers,Total number of Items Sold,Avg. Total Order Amount
shop_id,,,
42,31,51,235101.490196
78,45,46,49213.043478


#### Looks like there are couple of stores that stand out in particular, shops 42 and 78. Let's investigate both a little further

In [15]:
# a single shoe from shop 78 costs $25,725! I would guess this is a data entry error? So a single shoe should be $257.25
df[df['shop_id']==78][['order_amount','total_items']].describe()

,order_amount,total_items
count,46.000000,46.000000
mean,49213.043478,1.913043
std,26472.227449,1.029047
min,25725.000000,1.000000
25%,25725.000000,1.000000
50%,51450.000000,2.000000
75%,51450.000000,2.000000
max,154350.000000,6.000000


In [9]:
## Shop 42 summary stats
df[df['shop_id']==42][['order_amount','total_items']].describe()

,order_amount,total_items
count,51.000000,51.000000
mean,235101.490196,667.901961
std,334860.641587,951.308641
min,352.000000,1.000000
25%,352.000000,1.000000
50%,704.000000,2.000000
75%,704000.000000,2000.000000
max,704000.000000,2000.000000


#### Large purchase orders are placed in bulk as shown above there is an outlier when looking at the breakdown of total items purchased in a single order, 2000 shoes!The price of a single shoe is \\$352 at this particular store 

In [13]:
# Investigating shop 42
# We can see that there are a few repeat customers that have placed orders with store 42
# There are also a few large single orders outside of the most obvious which is provided by user_id 607 
## user_id 607 is both a repeat customer and also a customer that buys in large quantities per individual order as shown
fourty_two=df[df['shop_id']==42]
fourty_two.groupby('user_id').agg({
    'order_amount':'mean',
    'total_items':'mean',
    'created_at':'count'
    
}).rename(columns={'order_amount':'Avg Order Value','total_items':'Avg Number of Orders Placed',
                  'created_at':'Total Number of Orders'})

,Avg Order Value,Avg Number of Orders Placed,Total Number of Orders
user_id,,,
607,704000,2000.0,17
720,704,2.0,1
726,1056,3.0,1
736,704,2.0,1
739,528,1.5,2
744,352,1.0,1
747,704,2.0,1
756,704,2.0,1
770,352,1.0,2


## Summary 

#### Based on these findings it explains why AOV is probably not the best metric to use. The mean is extremely sensitive to skewed data and  this dataset is exactly that, skewed. Instead we should consider using Median order Value. This metric takes into account both of the situations we discovered which are varying price points across individual stores (or maybe data entry errors) and also large order quantities and is more robust to such outliers

In [7]:
aov=np.round((df['total_items']*np.mean(df['order_amount']))/(df['total_items']),2)[0]
median_order_value=np.round((df['total_items']*np.median(df['order_amount']))/(df['total_items']),2)[0]

print("Average order Value: ${} ".format(aov))
# our median order value is much more reasonable/realistic at $284 
print("Median order Value: ${}".format(median_order_value))


Average order Value: $3145.13 
Median order Value: $284.0


#### The median order value provides a new reasonable metric of \\$284.00 compared to the old \\$3,145.13 value